In [3]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import seaborn as sns
import os

from PIL import Image, ImageOps
import pytesseract
import numpy as np

from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
from typing import Iterable
import concurrent

from tqdm import tqdm

import keras_ocr

# Exploracion y pruebas con OCR sobre cada tipo de captcha

Tenemos 100 muestras de cada tipo de captcha, incluyendo el texto de entrenamiento, haremos la prueba para saber cuantos y cuales se pueden reconocer directamente

In [4]:
# las imagenes estan en la carpeta imagenes, y tienen el nombre en formato tipo--texto.png
# ejemplo: AncientMosaic--RXWVJ.png
# debemos crear un dataset con el tipo de la imagen, el texto que contiene, la imagen y el texto resultante del OCR y si es correcto o no



# helper para obtener los archivos de una carpeta
def get_files(directory: Path) -> Iterable[Path]:
    # Using glob simplifies the code in these cases.
    return (file for file in directory.glob("*.png") if file.is_file())

# helper para procesar cada archivo
def procesar(file: Path):
    # obtener el texto de la imagen
    name = file.name
    tipo = name.split('--')[0]
    texto = name.split('--')[1].split('.')[0]

    # leer la imagen y transformarla a gris
    imagen = Image.open(file)
    imagen = ImageOps.grayscale(imagen)
    # imagen = np.array(imagen)

    # leer el texto usando OCR
    texto_ocr = pytesseract.image_to_string(imagen)

    # devolver los datos procesados
    fila = [tipo, texto, imagen, texto_ocr, texto == texto_ocr]
    return fila


def process_files(directory: Path):
    # I am using sum, so that the result is computed lazily and we
    # do not need to build a list of all files. If the directory is
    # very large, this could save a lot of memory.
    # Since get_files returns a one-shot generator, we cannot
    # save it to a variable and reuse it here and below.
    file_count = sum(1 for _ in get_files(directory))
    # crear el dataframe
    columns=['tipo', 'texto', 'imagen', 'texto_ocr', 'correcto']
    df = pd.DataFrame(columns=columns)
    
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = executor.map(procesar, get_files(directory))
        # Reading the value returned by the executor is very
        # important, because exceptions happening in the `foo`
        # function will be captured and not be raised until the
        # value is read, thus obfuscating the errors.
        #        
        # Another nice solution for the progress would be
        # using the library tqdm.
        # for i, _ in enumerate(futures):
        #     print(f"Processed file {i+1} / {file_count}")
    
        for i, result in enumerate(tqdm(futures, total=file_count)):
            s = pd.Series(result, index=columns)  # crar fila

            # a ser posible siempre se agrega a un indice predefinido,
            # la alternativa es usar len(df) pero esto supone calcular la 
            # longitud en cada iteracion, lo que es muy lento
            df.loc[i] = s  # agregar fila
    
    return df

df = process_files(Path('imagenes2'))

100%|██████████| 19/19 [00:00<00:00, 98.27it/s]


In [23]:
from itertools import permutations, combinations

caracteres = [c for c in 'ABCDEF']
codigos = list(permutations(caracteres, 5))
len(codigos)
[''.join(x) for x in codigos]

['ABCDE',
 'ABCDF',
 'ABCED',
 'ABCEF',
 'ABCFD',
 'ABCFE',
 'ABDCE',
 'ABDCF',
 'ABDEC',
 'ABDEF',
 'ABDFC',
 'ABDFE',
 'ABECD',
 'ABECF',
 'ABEDC',
 'ABEDF',
 'ABEFC',
 'ABEFD',
 'ABFCD',
 'ABFCE',
 'ABFDC',
 'ABFDE',
 'ABFEC',
 'ABFED',
 'ACBDE',
 'ACBDF',
 'ACBED',
 'ACBEF',
 'ACBFD',
 'ACBFE',
 'ACDBE',
 'ACDBF',
 'ACDEB',
 'ACDEF',
 'ACDFB',
 'ACDFE',
 'ACEBD',
 'ACEBF',
 'ACEDB',
 'ACEDF',
 'ACEFB',
 'ACEFD',
 'ACFBD',
 'ACFBE',
 'ACFDB',
 'ACFDE',
 'ACFEB',
 'ACFED',
 'ADBCE',
 'ADBCF',
 'ADBEC',
 'ADBEF',
 'ADBFC',
 'ADBFE',
 'ADCBE',
 'ADCBF',
 'ADCEB',
 'ADCEF',
 'ADCFB',
 'ADCFE',
 'ADEBC',
 'ADEBF',
 'ADECB',
 'ADECF',
 'ADEFB',
 'ADEFC',
 'ADFBC',
 'ADFBE',
 'ADFCB',
 'ADFCE',
 'ADFEB',
 'ADFEC',
 'AEBCD',
 'AEBCF',
 'AEBDC',
 'AEBDF',
 'AEBFC',
 'AEBFD',
 'AECBD',
 'AECBF',
 'AECDB',
 'AECDF',
 'AECFB',
 'AECFD',
 'AEDBC',
 'AEDBF',
 'AEDCB',
 'AEDCF',
 'AEDFB',
 'AEDFC',
 'AEFBC',
 'AEFBD',
 'AEFCB',
 'AEFCD',
 'AEFDB',
 'AEFDC',
 'AFBCD',
 'AFBCE',
 'AFBDC',
 'AFBDE',


In [24]:
p = "\x49\155\141\147\x65\x20\107\145\156\x65\x72\x61\164\x6f\162\x20\x6e\x6f\x74\40\151\x6d\x70\x6c\x65\155\x65\156\x74\145\144\41"
p

'Image Generator not implemented!'